<a href="https://colab.research.google.com/github/NinaMaz/mlss-tutorials/blob/master/Adaboost2019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install -U scikit-learn 
!pip install matplotlib
!pip install seaborn
!pip install numpy
!pip install pandas

# Adaboost

## Boosting

The underlying idea of boosting is to combine **cheap weak**
predictiors into a strong **powerful ensemble**.

![](https://www.researchgate.net/profile/Eiji_Hato/publication/264713074/figure/fig5/AS:614118558023749@1523428682764/Concept-of-AdaBoost.png)

### AdaBoost

This algorithm uses a greedy strategy to construct an additive ensemble of size $T$
from a dictionary of basis predictors.

The size of the ensemble $T$ is a regularization parameter:
* the greater the $T$, the more boosting overfits.

**`AdaBoost.M1`**:

![alt text](https://i.imgur.com/im1cMeS.jpg)

#### AdaBoost in SciKit

The `ensemble` submodule implements both classification and regression
versions of Adaboost
* AdaBoostClassifier
* AdaBoostRegressor

Algorithms differ only in the **loss** used in determining the weight updates.

In [0]:
from sklearn.ensemble import AdaBoostClassifier, AdaBoostRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor

### Adaboost params

Common parameters:
- **n_estimators** -- the maximum size of the ensemble (in **case of a perfect fit** boosting stops earlier);
- **base_estimator** -- the base estimator of the ensemble, which must support **sample weighting**;
- **learning_rate** -- scales each update, slowing down convergence, but making it more accurate.

AdaBoostClassifier only:
- **algorithm** -- the AdaBoost version to use:
    * _"SAMME"_ -- the SAMME (**M1**) discrete boosting algorithm;
    * _"SAMME.R"_ -- the SAMME.R real boosting algorithm, typically achieves a lower test error with fewer boosting iterations than `SAMME`.
    
AdaBoostRegressor only:
- **loss** -- the loss function to use when updating the weights after each boosting iteration:
    * _"linear"_ -- absolute loss $L(y, p) = |y-p|$;
    * _"square"_ -- squared loss $L(y, p) = |y-p|^2$;
    * _"exponential"_ -- Exponential loss $L(y, p) = 1-e^{-|y-p|}$.

# Practice: toy example

In [0]:
from sklearn.datasets import make_gaussian_quantiles
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, accuracy_score, mean_absolute_error, mean_squared_error, f1_score
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import numpy as np
import pandas as pd

%matplotlib inline

random_state = 1

In [0]:
def scikit_example(n_samples, random_state=None):
    X1, y1 = make_gaussian_quantiles(cov=2., n_samples=int(0.4*n_samples),
                                     n_features=2, n_classes=2,
                                     random_state=random_state)
    X2, y2 = make_gaussian_quantiles(mean=(3, 3), cov=1.5,
                                     n_samples=int(0.6*n_samples),
                                     n_features=2, n_classes=2,
                                     random_state=random_state)
    return np.concatenate((X1, X2)), np.concatenate((y1, 1 - y2))

In [0]:
X, y = scikit_example(n_samples=500, random_state=random_state)

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.5, random_state=random_state)


min_, max_ = np.min(X, axis=0) - 1, np.max(X, axis=0) + 1
xx, yy = np.meshgrid(np.linspace(min_[0], max_[0], num=51),
                     np.linspace(min_[1], max_[1], num=51))

In [0]:
def plot_adaboost(adaboost=None):
    mpl.style.use('seaborn')
    colors = ['red','green','blue']

    ax = plt.figure(figsize=(12, 8)).add_subplot(111)
    ax.scatter(X_train[:, 0], X_train[:, 1], c=y_train, cmap=mpl.colors.ListedColormap(colors))

    if adaboost:
        prob_ = adaboost.predict_proba(np.c_[xx.ravel(), yy.ravel()])[:, 1].reshape(xx.shape)
        ax.contourf(xx, yy, prob_, alpha=0.4, cmap=plt.cm.coolwarm)
        ax.set_title("AdaBoost with %d trees"%(adaboost.n_estimators,));

In [0]:
plot_adaboost()

In [0]:
abc1_ = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5),
                          n_estimators=5, random_state=random_state)
abc1_.fit(X_train, y_train)


plot_adaboost(abc1_)

In [0]:
abc2_ = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5),
                          n_estimators=100, random_state=random_state)
abc2_.fit(X_train, y_train)

plot_adaboost(abc2_)

In [0]:
abc3_ = AdaBoostClassifier(DecisionTreeClassifier(max_depth=5),
                          n_estimators=1000, random_state=random_state)
abc3_.fit(X_train, y_train)

plot_adaboost(abc2_)

In [0]:
ax = plt.figure(figsize=(12, 8)).add_subplot(111)
for est_ in abc1_.estimators_:
    fpr, tpr, _ = roc_curve(y_test, est_.predict_proba(X_test)[:, 1])
    ax.plot(fpr, tpr, color="red", alpha=0.5)

fpr, tpr, _ = roc_curve(y_test, abc1_.predict_proba(X_test)[:, 1])
ax.plot(fpr, tpr, color="black", label="AdaBoost %d"%(abc1_.n_estimators,))

ax.legend(fontsize="xx-large")
ax.set_title("ROC curve for AdaBoost");

In [0]:
ax = plt.figure(figsize=(12, 8)).add_subplot(111)
for est_ in abc2_.estimators_:
    fpr, tpr, _ = roc_curve(y_test, est_.predict_proba(X_test)[:, 1])
    ax.plot(fpr, tpr, color="red", alpha=0.05)

fpr, tpr, _ = roc_curve(y_test, abc2_.predict_proba(X_test)[:, 1])
ax.plot(fpr, tpr, color="black", label="AdaBoost %d"%(abc2_.n_estimators,))

ax.legend(fontsize="xx-large")
ax.set_title("ROC curve for AdaBoost");

In [0]:
ax = plt.figure(figsize=(12, 8)).add_subplot(111)
for est_ in abc3_.estimators_:
    fpr, tpr, _ = roc_curve(y_test, est_.predict_proba(X_test)[:, 1])
    ax.plot(fpr, tpr, color="red", alpha=0.05)

fpr, tpr, _ = roc_curve(y_test, abc3_.predict_proba(X_test)[:, 1])
ax.plot(fpr, tpr, color="black", label="AdaBoost %d"%(abc3_.n_estimators,))

ax.legend(fontsize="xx-large")
ax.set_title("ROC curve for AdaBoost");

# Adult dataset


Download the dataset. If you use Windows, you can download it manually by the link [https://github.com/amueller/advanced_training/raw/master/data/adult.csv](https://github.com/amueller/advanced_training/raw/master/data/adult.csv)

In [0]:
!wget https://github.com/amueller/advanced_training/raw/master/data/adult.csv

Let's look at it. We have some information about different persons. Our target value will be the income column. Since we have the information whether the income is above or below 50K, we will solve binary classification problem.

In [0]:
data = pd.read_csv('adult.csv', index_col=0)
print(data.shape)
data.head()

In [0]:
size_before = data.shape[0]
(data.astype(str) == ' ?').any()

In [0]:
data = data[data.workclass != ' ?']
data = data[data.occupation != ' ?']
data = data[data['native-country'] != ' ?']

# data = data[np.logical_and(data.workclass != ' ?', data.occupation != ' ?')]

size_after = data.shape[0]

(data.astype(str) == '?').any()

In [0]:
data.shape

Let's conver our target income to binary value (0 and 1). 

In [0]:
if 'income' in data.columns:
    y = (data['income'] == ' >50K').values.astype(np.int32)
    del data['income']
    
del data['native-country']
    
y.shape

We have some categorical data in our dataset. Let us use one-hot-encoding for this columns. In order to automaticalli convert the data you should use method pd.get_dummies()

In [0]:
data = pd.get_dummies(data, columns=['workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'gender'])
data.head()

In [0]:
data.shape, y.shape

# should we scale our data before AdaBoost?

![](https://telegra.ph/file/db989d671719e18433b57.jpg)

In [0]:
from sklearn.preprocessing import scale

X = scale(data.astype(np.float))

!!! Don't change random state

In [0]:
X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.2, random_state=9, stratify=y)

X_val, X_test, y_val, y_test = \
    train_test_split(X_test, y_test, test_size=0.5, random_state=9, stratify=y_test)

In [0]:
X_train.shape, X_val.shape, X_test.shape

In [0]:
def get_test_train_errors(_adaboost):
    train_errors = []

    for y in _adaboost.staged_predict(X_train):
        train_errors.append(f1_score(y_train, y))

    test_errors = []

    for y in _adaboost.staged_predict(X_val):
        test_errors.append(f1_score(y_val, y))
        
    print('F1 validation score:', f1_score(y_val, _adaboost.predict(X_val)))
    
    plt.figure(figsize=(12, 8))
    plt.plot(train_errors, label='train')
    plt.plot(test_errors, label='test')
    plt.legend(loc='best')

## F1 score

Since we consider binary classification, it is good to use some proper measure. One of them — F1


$$F1 = 2 * (precision * recall) / (precision + recall)$$

![](https://upload.wikimedia.org/wikipedia/commons/thumb/2/26/Precisionrecall.svg/350px-Precisionrecall.svg.png)

Let's train AdaBoost with a lot of small trees

In [0]:
adaboost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1), n_estimators=1000)
adaboost.fit(X_train, y_train)

get_test_train_errors(adaboost)

Now, increase the depth of trees

In [0]:
adaboost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5), n_estimators=1000)
adaboost.fit(X_train, y_train)

get_test_train_errors(adaboost)

Leave trees depth as 5, decrease n_estimators to 200

In [0]:
adaboost = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5), n_estimators=200)
adaboost.fit(X_train, y_train)

get_test_train_errors(adaboost)

## Hello, overfitting!

![](https://upload.wikimedia.org/wikipedia/commons/thumb/1/19/Overfitting.svg/300px-Overfitting.svg.png)

# Early stopping

Ok, we understood, that we don't need to much trees: we overfit. But if we take not enough trees, we will overfit. Here comes the regularization.

And one of the best regularization methods (best in practice), that is applied to ensembles (and to NN) is early stopping.

## The idea behind it is simple: if our validation loss didn't increase last 'n_tolerance' steps, then we stop adding new trees.

![](https://deeplearning4j.org/images/guide/earlystopping.png)


Let's implement Early Stopping for adaboost algorithm. We will extend the class AdaBoostClassifier. In order to implement the Early Stopping, you should understand, that it is easier to cut the final algorithm (select only a number of first trees) instead of training it each time.

In [0]:
class EarlyStoppingAdaBoost(AdaBoostClassifier):
    def __init__(self, n_tolerance, score_func, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.n_tolerance = n_tolerance
        self.score_func = score_func
        
    def fit(self, X_train, y_train, X_val, y_val):
        # train big AdaBoost
        super().fit(X_train, y_train)
        
        # Now you have to write an early stopping algorithm
        ######################
        # YOUR CODE HERE!
        # YOUR CODE HERE!
        ######################
        
        return self

In [0]:
adaboost = EarlyStoppingAdaBoost(base_estimator=DecisionTreeClassifier(max_depth=5),
                                 n_estimators=200,
                                 n_tolerance=10, score_func=f1_score)
adaboost.fit(X_train, y_train, X_val, y_val)

get_test_train_errors(adaboost)

In [0]:
adaboost = EarlyStoppingAdaBoost(base_estimator=DecisionTreeClassifier(max_depth=1),
                                 n_estimators=1000,
                                 n_tolerance=10, score_func=f1_score)
adaboost.fit(X_train, y_train, X_val, y_val)

get_test_train_errors(adaboost)

# Feature Importance: how?

In [0]:
importances = adaboost.feature_importances_
std = np.std([tree.feature_importances_ for tree in adaboost.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]

print("Feature ranking:")

for f in range(X.shape[1]):
    print("%d. feature %s (%f)" % (f + 1, data.columns[f], importances[indices[f]]))

# Stacking: Voting

Imagine you have two powerful algorithms, such as SVM and AdaBoost. You found the best hyperparameters for each of the algorithms. Usually in practice you can increase the prediction quality by combining two algorithms using stacking.

One of the way to perform stacking is voting. If you have two algorithms $clf_1(x)$ and $clf_2(x)$, then the prediction will be 

$$result(x) = clf_1(x) * \alpha + clf_2(x) * (1 - \alpha),$$

where $\alpha \in [0, 1]$

In [0]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.model_selection import cross_val_score

In [0]:
clf1 = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                                 n_estimators=20)
clf2 = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3),
                                 n_estimators=50)


mix_clf_scores = []
for alpha in np.linspace(0., 1., 10):
    clf = VotingClassifier([('clf1', clf1), ('clf2', clf2)], 'soft', [alpha, 1-alpha])
    clf.fit(X_train, y_train)
    
    f1 = f1_score(y_val, clf.predict(X_val))
    
    mix_clf_scores.append(f1)


In [0]:
plt.plot(np.linspace(0., 1., 10), mix_clf_scores)
plt.xlabel('alpha')
plt.ylabel('f1_score')
plt.title('VotingClassifier scores depending from weights')

# Stacking: Meta-learner

![](https://blogs.sas.com/content/subconsciousmusings/files/2017/05/modelstacking.png)

Another possible solution for combination of several algorithms: learning some meta algorithm on the top of some base algorithms. For example, you have trained such algorithms, as SVM, Random Forest and Adaboost on your dataset. Each of these algorithms is able to return some scores, that you can use as features of the input objects. After it you can train some easy linear model (linear regression, logistic regression) on this features (which are the output of base algorithms).

## What we can use as the information from the base algorithms?

![](https://telegra.ph/file/f8b9d813663403520cfc6.jpg)

If you would like to construct such kind of stacking, you should be aware of overfitting. If you use the same train set for both base learners and meta learners, you will overfit.



So, your task for now is to implement stacking. You can choose what to do yourself:

* Use K-fold stacking
* Split your data into train and calibration sets
* Use train set for both training and calibration, and then obtain overfitting



In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_predict
from sklearn.base import clone

class MetaLogisticRegression(LogisticRegression):
    def __init__(self, base_learners, penalty='l2', dual=False, tol=0.0001,
                 C=1.0, fit_intercept=True, intercept_scaling=1,
                 class_weight=None, random_state=None, solver='liblinear',
                 max_iter=100, multi_class='ovr', verbose=0,
                 warm_start=False, n_jobs=None):
        super().__init__(penalty, dual, tol, C, fit_intercept, intercept_scaling,
                         class_weight, random_state, solver, max_iter, multi_class,
                         verbose, warm_start, n_jobs)
        self.base_learners = base_learners
        
        
    def fit_predict_base_learners(self, _X, _y):
        # write a function, that will feat each base classiffier and
        # return predictions of base learners for the train set (M1, M2 in algorithm)
        # HINT: from sklearn.model_selection import KFold
        # https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html
        ######################
        # YOUR CODE HERE!      
        # YOUR CODE HERE!
        ######################
        
    def fit_base_learners_full(self, _X, _y):
        # write a function to fit a base learners on the whole(!) train set
        ######################
        # YOUR CODE HERE!
        # YOUR CODE HERE!
        ######################
        
    def get_base_predictions(self, _X):
        # write a function, get the predictions of each base algorithm and return it
        # in the form of matrix of shape [X.shape[0], n_base_algorithms]
        ######################
        # YOUR CODE HERE!
        # YOUR CODE HERE!
        ######################
        
    def fit(self, _X, _y):
        # K-fold fitting of base learners
        base_predictions = self.fit_predict_base_learners(_X, _y)
        
        # learn meta Logistic Regression
        super().fit(base_predictions, _y)
        
        # fitting for test predictions
        self.fit_base_learners_full(_X, _y)        
        return self
    
    def predict(self, _X):
        base_predictions = self.get_base_predictions(_X)
        return super().predict(base_predictions)

In [0]:
clf1 = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=5),
                                 n_estimators=20)
clf2 = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=3),
                                 n_estimators=50)

meta = MetaLogisticRegression(base_learners=[clf1, clf2])
meta.fit(X_train, y_train)

In [0]:
f1_score(y_val, meta.predict(X_val))

# Snickers Challenge

Now let's check your imagination in designing ML algorithms. You task is to construct an algorithm in order to achive the highest f1 score on the dataset we considered today.

The winner will obtain:

![](https://www.utkonos.ru/resample/450x450q95/images/photo/3220/3220214H.jpg?mtime=5bb36059)

You have X_train, X_val, X_test. You can train/validate on both X_train and X_val on your choice. X_test — this is only for test prediction.

Key ingredients to success in Kaggle:
* Boosting
* Stacking
* Feature Engineering (removing non-important features, constructing new features)
* Data preprocessing (outliers)

You may start with stacking of several algorithms of different nature.

!!! Share your score here: https://goo.gl/forms/iINp60KzOwilIk6p1

In [0]:
def get_test_score(y_pred):
    return f1_score(y_pred, y_test)